In [2]:
import csv
import copy
import string
import random
import heapq
import pandas as pd
import numpy as np
import pygad
import time
import math
from collections import defaultdict

In [3]:
def set_tasks(filename, N = 8, num_sol = 2):
    '''
    filename (str): the file that contains start and goal states 
    N (int): size of deck 
    '''

    start_states = [] # a list of matrices
    start_dicts = []
    goal_states = [] # a list of dict{col: [sol_units, sol_units, ..., ]}
    with open(filename, 'r') as f:
        lines = [line.strip().lower() for line in f.readlines()]
    def default_value():
                return np.zeros(num_sol)
    for i in range(len(lines)):
        if lines[i].startswith('start'):
            start = np.zeros((num_sol, N * N))
            start_dict = defaultdict(default_value)
            j = i + 1
            while not lines[j].startswith('end'):
                x, y, sol = map(int, lines[j].split())
                col = x * N + y
                start[sol, col] += 1
                start_dict[col][sol] += 1
                j += 1
            start_states.append(start)
            start_dicts.append(start_dict)
        elif lines[i].startswith('goal'):
            goal = defaultdict(default_value)
            j = i + 1
            while not lines[j].startswith('end'):
                x, y, sol = map(int, lines[j].split())
                col = x * N + y
                goal[col][sol] += 1
                j += 1
            goal_states.append(goal)
        
            
    return start_states, goal_states, start_dicts


In [4]:
class Machine:
    def __init__(self):
        pass
    
    def BeamSearch(self, start, goal, beam_size = 10, N = 5, head_pos = (0, 0), tip_cap = 100, well_cap = 500, num_sol = 2):
        self.N = N
        self.head_pos = head_pos
        self.tip_cap = tip_cap
        self.well_cap = well_cap
        self.num_sol = num_sol
    
        # Construct status matrix (row as solution, column as well)
        head = np.zeros((self.num_sol, 1))
        #print(head)
        self.status = np.hstack((start, head))
        self.goal = copy.deepcopy(goal)
        
        # run beamsearch algorithm
        s = time.time()
        self.solutions = self.beam_search(self.status, beam_size)      
        t = time.time()
        self.timing = t - s
        
        # Uncomment the code below to display the result
        '''
        print()
        print('Job Completed in', self.timing)
        for i, final_beam in enumerate(self.solutions):
            print('------solution', i, ', cost:', final_beam.cost)
            print(': score =', final_beam.score[0] + final_beam.score[1] + 0.2 * final_beam.score[2] + 0.01* final_beam.score[3])
            print(final_beam.protocol)
        '''
        
            
    class Beam:
        def __init__(self, score, cost, status, head_pos, protocol, visit_times):
            # score is a 4-element list, each represents a perspective of operation estimation function
            self.score = copy.deepcopy(score)
            self.cost = cost
            self.status = status
            self.head_pos = head_pos
            self.protocol = protocol
            self.visit_times = copy.deepcopy(visit_times)
            self.manipulated = self.unique_wells_manipulated()

        def unique_wells_manipulated(self):
            manipulated = []
            for col in self.visit_times.keys():
                if self.visit_times[col] > 0:
                    manipulated.append(col)
            return manipulated
        
        def print_(self):
            print()
            print('        beam total score: ', np.sum(self.score))
            print('        C_diff, rmsd, unique_manipulated, cost: ', self.score)
            print('        beam cost:', self.cost)
            print('        beam head pos:', self.head_pos)
            print('        beam protocol:', self.protocol)
            print('        manipulated unique wells:', self.manipulated)
            print('        number of manipulated uniques', len(self.manipulated))
            print()

    def beam_search(self, curr_status, beam_size, mc = 1, ac = 0, dc = 0, zc = 0):
        self.mc = mc
        self.ac = ac
        self.dc = dc
        self.zc = zc
        status = copy.deepcopy(curr_status)
        
        # Initialize the beam with a list that contains one Beam instance:
        # Beam(score = [], cost = 0, status, head_pos = self.head_pos, protocol = [], visit_times = dict(int) maniputed wells with number of visited times)
        init_beam = self.Beam([], 0, status, self.head_pos, [], defaultdict(int))
        beams = [init_beam] 
        solutions = []
        iter_num = 0
        
        # Search until we found beam_size number of protocols that achieve the goal plate, or reach to maximum iterations
        while len(solutions) < beam_size and iter_num < 200:
            new_beams = []
            
            for beam in beams:
                candidates = self.transition_func(beam, beam_size)
                new_beams.extend(candidates)
            #normalize score of new beams and sort by the their sum
            score_Arr = [new_beam.score for new_beam in new_beams]
            norms = np.linalg.norm(score_Arr, axis = 0)
            score_Arr = score_Arr / norms
            #Update the normalized variable in each class instance
            for i, new_beam in enumerate(new_beams):
                new_beam.score = score_Arr[i, :]
            new_beams = sorted(new_beams, key = lambda x: x.score[0] + x.score[1] + 0.08* x.score[2] + 0.08 * x.score[3] + 0.1 * x.score[4])
            k = min(beam_size, len(new_beams))
            new_beams = new_beams[:k]
            beams = []
            for beam in new_beams:
                if self.is_goal_status(beam):
                    solutions.append(beam)
                else:
                    beams.append(beam)
                    
            if len(beams) == 0 and len(solutions) < beam_size:
                print('        Run out of solutions. Terminate Task.', len(solutions), ' solutions found.')
                return solutions
            iter_num += 1
        return solutions
    
    def transition_func(self, beam, beam_size):
        '''
        return the best beam_size number of next commands regarding to given status,
        as well as corresponding cost, next status, and head position.
        '''
        # list of tuple(cost, cmd, next_status, next_head_pos) to be returned
        candidates = [] 
        # get total units of sol in head tip
        head_vol = np.sum(beam.status[:, -1]) 
        head_C = beam.status[:, -1] / head_vol
        
        for i in range(beam.status.shape[1] - 1):
            well_vol = np.sum(beam.status[:, i])
            well_C = beam.status[:, i] / well_vol

            if i in self.goal.keys():
                # get concentration of each solution
                goal_vol = np.sum(self.goal[i])
                goal_C = self.goal[i] / goal_vol 
                
                # only set commands to the goal well under situations below
                cmd = None
                if np.any(well_C != goal_C):
                    if well_vol == 0:
                        # dsp only
                        if np.all(head_C == goal_C):
                            cmd = (2, i, min(head_vol, goal_vol))
                    else:
                        # asp only 
                        cmd = (1, i, min(well_vol, self.tip_cap - head_vol))
                        #print('cmd given by goal', cmd)
                else:
                    if well_vol > goal_vol:
                        cmd = (1, i, min(well_vol - goal_vol, self.tip_cap - head_vol))
                        #print('cmd given by goal', cmd)
                    else:
                        if np.all(head_C == goal_C):
                            cmd = (2, i, min(goal_vol - well_vol, head_vol))
                
                if cmd != None:
                    beam_cand = self.beam_after_cmd(cmd, beam, i)
                    candidates.append(beam_cand)
                    
                            
            # for wells not specified in goals
            else:
                if well_vol == 0: 
                    if head_vol > 0:
                        # zero
                        cmd = (0, i, head_vol)
                        beam_cand = self.beam_after_cmd(cmd, beam, i)
                        candidates.append(beam_cand)
                        
                        # dsp
                        cmd = (2, i, min(self.well_cap, head_vol))
                        beam_cand = self.beam_after_cmd(cmd, beam, i)
                        candidates.append(beam_cand)
                else: 
                    if head_vol > 0:
                        if np.all(head_C == well_C):
                            # dsp
                            cmd = (2, i, min(self.well_cap - well_vol, head_vol))
                            beam_cand = self.beam_after_cmd(cmd, beam, i)
                            candidates.append(beam_cand)
                            
                            if head_vol < self.well_cap - well_vol:
                                # zero
                                cmd = (0, i, head_vol)
                                beam_cand = self.beam_after_cmd(cmd, beam, i)
                                candidates.append(beam_cand)
                    # asp
                    cmd = (1, i, min(well_vol, self.tip_cap))
                    #print('cmd not given by goal', cmd)
                    beam_cand = self.beam_after_cmd(cmd, beam, i)
                    candidates.append(beam_cand)

        return candidates
    
    def e1(self, beam):
        '''
        obtained from 4 perpectives:
        1. concentration difference (goal wells only)
        2. rmsd (goal wells only)
        3. number of unique wells that have been manipulated, weighted by number of operations
           (the higher the number, the larger the cost)
        4. cost
        5. number of times the well to be operated that has been manipulated 
        normalization will be applied after all possible operations are found out
        '''
        C_diff = 0
        rmsd = 0
        #print('            beam evaluated:', beam.protocol)
        for col, sol in self.goal.items():
            goal_vol = np.sum(self.goal[col]) # wells specified in goal always have goal_vol > 0
            goal_C = self.goal[col] / goal_vol 
            #beam.print_()
            well_vol = np.sum(beam.status[:, col])
            well_C = 0
            if well_vol != 0:
                well_C = beam.status[:, col] / well_vol
            C_diff += np.sum(abs(well_C - goal_C))
            #print('            well at goal, well at beam:', self.goal[col], beam.status[:, col])
            rmsd += np.sum(abs(self.goal[col] - beam.status[:, col])) ** 2  
            #print('            well, rmsd:', col, rmsd)
            
        rmsd = math.sqrt(rmsd / len(self.goal.keys()))
        unique_manipulated = len(beam.manipulated) # weighted by number of operations
        num_ops = sum(beam.visit_times.values())
        
        cmd_col = beam.protocol[-1][1]
        #print('command col', cmd_col)
        well_repeat = beam.visit_times[cmd_col]
        
        score = np.array([C_diff, rmsd, num_ops * unique_manipulated, well_repeat, beam.cost])
        beam.score = score
        
            
    def beam_after_cmd(self, cmd, beam, col):
        #print('                 beam:', beam.protocol)
        next_cost, next_status, next_head_pos = self.simulate_cmd(cmd, beam.head_pos, beam.status)
        next_protocol = copy.deepcopy(beam.protocol)
        next_protocol.append(cmd)
        new_visit_times = copy.deepcopy(beam.visit_times)
        new_visit_times[col] += 1
        
        #score, cost, status, head_pos, protocol, ng_touched, g_touched
        beam_cand = self.Beam([], beam.cost + next_cost, next_status, next_head_pos, next_protocol, new_visit_times)
        self.e1(beam_cand)
        return beam_cand
    
    def simulate_cmd(self, command, head_pos, status):
        cost = 0
        op, col, vol = command
        next_status = copy.deepcopy(status)
        
        x, y = col // self.N, col % self.N
        next_head_pos = (x, y)
        
        move_dst = abs(x - head_pos[0]) + abs(y - head_pos[1])
        cost += move_dst * self.mc 
        if op == 0: #zero, move to y, tip - 
            cost += self.zc
            next_status[:, -1] = 0
        elif op == 1: #asp, move to y, well - , tip + 
            cost += self.ac
            C = status[:, col] / np.sum(status[:, col])
            vol = vol * C
            next_status[:, col] -= vol
            next_status[:, -1] += vol
        elif op == 2: #dsp, move to y, well + , tip -
            cost += self.dc
            C = status[:, -1] / np.sum(status[:, -1]) # concentration of each solution of head tip
            vol = vol * C
            next_status[:, col] += vol
            next_status[:, -1] -= vol
        else:
            print("operation is not recognized")
            return
        
        return cost, next_status, next_head_pos

    # PASSED
    def is_goal_status(self, beam):
        if beam.score[0] == 0 and beam.score[1] == 0:
            return True
        return False

In [ ]:
# example code to run BeamSearch with given goal configuration file, beam size, and number of solutions
if __name__ == '__main__':
    nan_count = 0
    to_check = []
    duration = 0
    total_move = 0
    
    '''
    change following three lines of code to run different experiments
    '''
    goal_file = 'tests_8x8_[4,4]_sp.txt'
    beam_size = 25
    num_sol = 2
    deck_size = 8
    outputfile = 'BS_output.txt'
    total_instances = 100
    start_states, goal_states, start_dicts = set_tasks(goal_file)
    
    with open(outputfile, 'w') as f:
        for i, (start, goal) in enumerate(zip(start_states, goal_states)):
            print('\n----------JOB', i, '----------')
            lhpo = Machine()
            lhpo.BeamSearch(start, goal, beam_size = beam_size, N = deck_size, num_sol = num_sol)
            line1 = 'RESULT ' + str(i) + '\n'
            f.write(line1)

            result = None
            # get the protocol with lowest cost
            if len(lhpo.solutions) > 0:
                print(len(lhpo.solutions), 'solutions found.')
                result = sorted(lhpo.solutions, key = lambda x: x.cost)[0]
                cost = 'cost ' + str(result.cost) + '\n'
                total_move += result.cost
                f.write(cost)
                timing = 'time ' + str(lhpo.timing) + '\n'
                duration += lhpo.timing
                f.write(timing)

                protocol = result.protocol
                np.savetxt(f, protocol, fmt = '%d')
            else:
                nan_count += 1
                to_check.append(i)
                cost = 'cost ' + 'Nan' + '\n'
                f.write(cost)
                timing = 'time ' + str(lhpo.timing) + '\n'
                duration += lhpo.timing
                f.write(timing)
                print('No solution found.')
            f.write('END\n')
   
    duration = duration / total_instances
    move = total_move / (total_instances - nan_count)
    success_rate = (total_instances - nan_count) / total_instances
    
    print('Average running time (sec):', duration)
    print('Average moving cost:', move)
    print('Success rate', success_rate)


----------JOB 0 ----------


<ipython-input-4-684a62d80ff1>:117: RuntimeWarning: invalid value encountered in true_divide
  head_C = beam.status[:, -1] / head_vol
<ipython-input-4-684a62d80ff1>:121: RuntimeWarning: invalid value encountered in true_divide
  well_C = beam.status[:, i] / well_vol
